Evaluate the predictions of all methods in the test set

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pickle
import networkx as nx
import nltk
import pandas as pd
import seaborn as sns
from sklearn.metrics import f1_score

In [2]:
# delimitar nombre y ubicacion del grafo a tomar para los subjects reales
carpeta_datos =  '.\\Data\\nx_data\\'
nombre_grafo = "genealogy_nx_graph.json"

In [3]:
# delimitar el nombre de la carpeta donde se tienen las predicciones
predictions_path = ".//Predictions//"

# delimitar el nombre del arhivo con la division de nodos
file_division_nodos =  "./Data/node_division.json"

In [4]:
# para cada metodo delimitar el nombre del archivo de predicciones
nombre_archivo_predicciones = {
    "Naive Bayes" : "Naive_Bayes.pkl",
    "BERT": 'BERT.pkl',
    "GCN": 'GCN.pkl',
    "Label Propagation": 'Label_Propagation.pkl',
    "Our_model_ML": 'probabilistic_graph_model_ML.pkl',
    "Our_model_MAP": 'probabilistic_graph_model_MAP.pkl'
}

In [5]:
# tomar los distintos metodos
metodos = list(nombre_archivo_predicciones.keys())

print(f"Se consideran los {len(metodos)} metodos:")
print(metodos)

Se consideran los 6 metodos:
['Naive Bayes', 'BERT', 'GCN', 'Label Propagation', 'Our_model_ML', 'Our_model_MAP']


## Leer grafo con subjects reales

In [6]:
# leer el grafo
with open(carpeta_datos + nombre_grafo, 'rb') as f:
    grafo = pickle.load(f)
    
# responder preguntas basicas
print(f"Se tienen {grafo.number_of_nodes()} matematicos")

Se tienen 297377 matematicos


In [7]:
# poner los subjects para los matematicos para los que se sepa
# esta es la ground truth
dict_ground_truth = {id_matematico: info_matematico['subject']
                     for id_matematico, info_matematico in grafo.nodes(data=True)
                     if info_matematico['subject'] is not None
                     and info_matematico['thesis'] is not None}

print(f"Se tiene ground truth para {len(dict_ground_truth)} matematicos")

Se tiene ground truth para 174501 matematicos


## Leer division de nodos

In [8]:
# division de nodos
with open(file_division_nodos, 'rb') as f:
    division_nodos = json.load(f)

In [9]:
# obtener distintos subconjuntos de ids
ids_train = set(division_nodos['nodos_train'])
ids_val = set(division_nodos['nodos_val'])
ids_test = set(division_nodos['nodos_test'])

# juntar estos conjuntos
ids_train_val_test = ids_train | ids_val | ids_test

In [10]:
# ver tamaños

print(f"Train ids: {len(ids_train)} nodos")
print(f"Val ids:   {len(ids_val)} nodos")
print(f"Test ids:  {len(ids_test)} nodos")
print(f"\nEn total se tienen {len(ids_train_val_test)} ids en train/val/test")

Train ids: 141345 nodos
Val ids:   15705 nodos
Test ids:  17451 nodos

En total se tienen 174501 ids en train/val/test


## Leer predicciones

In [11]:
# funcion para leer un archivo pickle
def load_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

In [12]:
# por cada metodo leer el diccionario de predicciones
diccionarios_predicciones = {
    metodo: load_pickle_file(predictions_path + nombre_predicciones)
    for metodo, nombre_predicciones in nombre_archivo_predicciones.items()
}

## Preparar comparaciones

In [13]:
# calcular metricas
def calcular_metricas(y_true, y_pred):
    '''
    Funcion auxuliar para calcular metricas de unas predicciones
    '''
    
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    f1_micro = f1_score(y_true, y_pred, average='micro')
    
    return {"f1_macro" :f1_macro, "f1_weighted" :f1_weighted, "f1_micro" :f1_micro}

In [14]:
# comparar todos los metodoe en un conjunto de ids
def comparar_predicciones_conjunto_ids(ids_interes):
    '''
    Argumentos:
    ids_interes - conjunto de ids de matematicos
    
    Se hace una comparacion de las predicciones de los distintos metodos
    en los matematicos con ids del conjunto de interes
    '''
    
    print(f"Prediccion en {len(ids_interes)} nodos")
    
    # por cada metodo poner un array con las predicciones de interes
    arrays_predicciones_interes = {
        metodo: np.array([dict_predicciones_metodo[n] for n in ids_interes])
        for metodo, dict_predicciones_metodo in diccionarios_predicciones.items()
    }

    # poner los verdaderos subjects de estos matematicos
    array_ground_truth_interes = np.array([dict_ground_truth[n] for n in ids_interes])
    
    # por cada metodo calcular las estadisticas de sus predicciones
    estadisticas_predicciones = {
        metodo: calcular_metricas(array_ground_truth_interes,
                                 array_predicciones_interes_metodo)
        for metodo, array_predicciones_interes_metodo in arrays_predicciones_interes.items()
    }
    
    # convertir en un df
    df_estadisticas = pd.DataFrame.from_dict(estadisticas_predicciones, orient='index')
    
    # ordenar de acuerdo a f1
    df_estadisticas = df_estadisticas.sort_values("f1_macro", ascending = False)
    
    return df_estadisticas

# Hacer las comparaciones

### Train

In [15]:
# todos los nodos en train
df_stats_train = comparar_predicciones_conjunto_ids(ids_train)
df_stats_train.round(4)

Prediccion en 141345 nodos


,f1_macro,f1_weighted,f1_micro
Our_model_ML,1.0000,1.0000,1.0000
Our_model_MAP,1.0000,1.0000,1.0000
Naive Bayes,0.9680,0.9691,0.9690
BERT,0.8589,0.9302,0.9321
GCN,0.8225,0.9010,0.9023
Label Propagation,0.7853,0.8592,0.8595


### Val

In [16]:
# en val todos los nodos
df_stats_val = comparar_predicciones_conjunto_ids(ids_val)
df_stats_val.round(4)

Prediccion en 15705 nodos


,f1_macro,f1_weighted,f1_micro
GCN,0.5689,0.7471,0.7509
Our_model_ML,0.5485,0.7353,0.7336
Our_model_MAP,0.5350,0.7382,0.7475
BERT,0.5006,0.6643,0.6688
Label Propagation,0.4801,0.6768,0.6569
Naive Bayes,0.4556,0.6422,0.6446


### Test

In [17]:
# en test todos los nodos
df_stats_test = comparar_predicciones_conjunto_ids(ids_test)
df_stats_test.round(4)

Prediccion en 17451 nodos


,f1_macro,f1_weighted,f1_micro
Our_model_ML,0.5693,0.7378,0.7361
GCN,0.5689,0.7370,0.7405
Our_model_MAP,0.5493,0.7373,0.7463
BERT,0.5028,0.6647,0.6695
Label Propagation,0.4812,0.6745,0.6562
Naive Bayes,0.4726,0.6417,0.6436
